In [36]:
import cv2
import pyarma as arma
import os
import numpy as np

## Temp

In [2]:
probe_path = "D:\\Dataset\\Silhouette_090-00\\00005"

In [64]:
import pickle

# write list to binary file
def save_list(a_list, path):
    # store list in binary file so 'wb' mode
    with open(path, 'wb') as fp:
        pickle.dump(a_list, fp)

# Read list to memory
def read_saved_list(path):
    # for reading also binary mode is important
    with open(path, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list


In [40]:
img = cv2.imread("0021.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [41]:
img.shape

(960, 1280)

In [43]:
simg = cv2.resize(img, (320, 240))

In [44]:
simg.shape

(240, 320)

In [45]:
cv2.imshow("simg", simg)
cv2.waitKey(0)


-1

## Util Functions

In [7]:
def getMean(tempX):
    csum = 0
    if len(tempX) <= 0:
        return csum
    for i in tempX:
        csum += i
    mean = csum // len(tempX)
    return mean

In [8]:
def findnonzero(mat):
    temp = []
    tempX = set()
    tempY = set()
    for i in range(mat.n_rows):
        for j in range(mat.n_cols):
            if mat[i,j] != 0:
                tempX.add(i)
                tempY.add(j)
    temp.append(tempX)
    temp.append(tempY)
    return temp

In [9]:
def findnonzeroM(mat):
    temp = []
    tempX = []
    tempY = []
    for i in range(mat.n_rows):
        for j in range(mat.n_cols):
            if mat[i,j] != 0:
                tempX.append(i)
                tempY.append(j)
    temp.append(tempX)
    temp.append(tempY)
    return temp

In [10]:
def getMaxValue(mat):
    mx = 0
    for i in mat:
        if len(i) > mx:
            mx = len(i)
    return mx

In [11]:
def cvtoarma(cv_mat_in):
    x = cv_mat_in.tolist()
    arma_mat = arma.mat(x)
    return arma_mat

In [12]:
def dealNor(mat):
    mx = getMaxValue(mat)
    temp = []
    size = 0
    for i in range(mx):
        value = 0.0
        for j in range(len(mat)):
            size = len(mat[j])
            if (i + 1) > size:
                mat[j].append(0.0)
            else:
                value += float(mat[j][i])
        temp.append(value)
    return temp

In [13]:
def write_list(lst, path):
    with open(path, 'w') as fp:
        for item in lst:
            # write each item on a new line
            fp.write("%s\n" % item)

def read_list(path):
    with open(path, 'r') as fp:
        lst = fp.read().splitlines()
        lst = list(map(float, lst))
        return lst

In [32]:
def calcDistance(sub_vcetors, probe_vectors):
	dis = 0.0
	usRowW = sub_vcetors[0]
	dis += calcCellDis(usRowW, probe_vectors[0])

	usColW = sub_vcetors[1]
	dis += calcCellDis(usColW, probe_vectors[1])

	vsRowW = sub_vcetors[2]
	dis += calcCellDis(vsRowW, probe_vectors[2])

	vsColW = sub_vcetors[3]
	dis += calcCellDis(vsColW, probe_vectors[3])

	return dis


def calcCellDis(sub, probe):
	usCurrentR = 0.0
	usSourceR = 0.0
	dis = 0.0
	for i in range(len(sub)):
		usCurrentR = sub[i]
		if (i > len(probe) - 1):
			usSourceR = 0.0
		else:
			usSourceR = probe[i]
		dis += abs(usCurrentR - usSourceR)
	return dis

In [16]:
def write_vectors(vecs, id):
    exists = os.path.exists('Gallery//' + str(id))
    usRowW = vecs[0]
    usColW = vecs[1]
    vsRowW = vecs[2]
    vsColW = vecs[3]
    if not exists:
        os.makedirs('Gallery//' + str(id))

    write_list(usRowW, 'Gallery//' + str(id) + '//usRowW.txt')
    write_list(usColW, 'Gallery//' + str(id) + '//usColW.txt')
    write_list(vsRowW, 'Gallery//' + str(id) + '//vsRowW.txt')
    write_list(vsColW, 'Gallery//' + str(id) + '//vsColW.txt')
    

## Functions for optical flow calculation

In [46]:
def make_sequence(path):
    sequence = []
    for i in os.scandir(path):
        if i.path.endswith(".png") and i.is_file():
            img = cv2.imread(i.path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # For OSAKA
            img = cv2.resize(img, (320, 240))
            # REMOVE Above line for CASIA
            arma_mat = cvtoarma(img)
            mean = getMean(findnonzero(arma_mat)[1]) # get mean of nonzero elements
            arma_mat = arma.shift(arma_mat, 160 - mean, 1)
            sequence.append(arma_mat)
    return sequence

In [15]:
def findFlowUtil(flow):
    usRow = []
    usCol = []
    vsRow = []
    vsCol = []
    for i in range(len(flow)):
        mus = flow[i][0]
        mvs = flow[i][1]
        MUSrowAndCol = findnonzero(mus)
        MVSrowAndCol = findnonzero(mvs)
        MUSTOTALrowAndcol = findnonzeroM(mus)
        MVSTOTALrowAndcol = findnonzeroM(mvs)

        usRvalue = 0.0
        usRtemp = []
        for usi in MUSrowAndCol[0]:
            for usii in MUSTOTALrowAndcol[0]:
                if usi == usii and mus[usi, usii] < 0.0:
                    usRvalue += mus[usi, usii]
            usRtemp.append(-usRvalue)
        usRow.append(usRtemp)

        usCvalue = 0.0
        usCtemp = []
        for usi in MUSrowAndCol[1]:
            for usii in MUSTOTALrowAndcol[1]:
                if usi == usii and mus[usi, usii] < 0.0:
                    usCvalue += mus[usi, usii]
            usCtemp.append(-usCvalue)
        usCol.append(usCtemp)

        vsRvalue = 0.0
        vsRtemp = []
        for vsi in MVSrowAndCol[0]:
            for vsii in MVSTOTALrowAndcol[0]:
                if vsi == vsii and mus[vsi, vsii] < 0.0:
                    vsRvalue += mus[vsi, vsii]
            vsRtemp.append(-vsRvalue)
        vsRow.append(vsRtemp)

        vsCvalue = 0.0
        vsCtemp = []
        for vsi in MVSrowAndCol[1]:
            for vsii in MVSTOTALrowAndcol[1]:
                if vsi == vsii and mvs[vsi, vsii] < 0.0:
                    vsCvalue += mvs[vsi, vsii]
            vsCtemp.append(-vsCvalue)
        vsCol.append(vsCtemp)
    
    usRowW = dealNor(usRow)
    usColW = dealNor(usCol)
    vsRowW = dealNor(vsRow)
    vsColW = dealNor(vsCol)
    return [usRowW, usColW, vsRowW, vsColW]

    

In [17]:
def findOpticalflow(sequence):
    flow = []
    out = []
    for i in range(len(sequence) - 1):
        f1 = sequence[i]
        f2 = sequence[i + 1]

        v1 = arma.mat(f1.n_rows, f1.n_cols)
        v1.zeros()
        v2 = arma.mat(f1.n_rows, f1.n_cols)
        v2.zeros()
        fx = arma.mat(f1.n_rows, f1.n_cols)
        fx.zeros()
        fy = arma.mat(f1.n_rows, f1.n_cols)
        fy.zeros()
        fz = arma.mat(f1.n_rows, f1.n_cols)
        fz.zeros()

        alpha2 = 625
        imax = 20

        filter1 = arma.mat(2, 2)
        filter1[0, 0] = -1
        filter1[0, 1] = 1
        filter1[1, 0] = -1
        filter1[1, 1] = 1
        filter1 = filter1 * 0.25
        temp1 = arma.conv2(f1, filter1, "same")
        temp2 = arma.conv2(f2, filter1, "same")
        fx = temp1 + temp2

        filter11 = arma.mat(2, 2)
        filter11[0, 0] = -1
        filter11[0, 1] = -1
        filter11[1, 0] = 1
        filter11[1, 1] = 1
        filter11 = filter11 * 0.25
        temp11 = arma.conv2(f1, filter11, "same")
        temp22 = arma.conv2(f2, filter11, "same")
        fy = temp11 + temp22

        filter111 = arma.mat(2, 2)
        filter11[0, 0] = 1
        filter111[0, 1] = 1
        filter111[1, 0] = 1
        filter111[1, 1] = 1
        filter111 = filter111 * 0.25
        temp111 = arma.conv2(f1, filter111, "same")
        temp222 = arma.conv2(f2, filter111, "same")
        fz = temp111 + temp222

        for i in range(imax):
            mat1 = fx @ v1
            mat2 = fy @ v2
            mat3 = mat1 + mat2
            mat3 = mat3 + fz

            mat4 = fx @ fx
            mat5 = fy @ fy
            mat6 = mat4 + mat5 
            mat6 = mat6 + alpha2

            delta = mat3 / mat6

            v1 = v1 - fx @ delta
            v2 = v2 - fy @ delta
        
        u = v1
        v = v2
        d = round(f1.n_rows / 50.0)
        hs_size = f1.n_rows // d
        ws_size = f1.n_cols // d
        us = arma.mat(hs_size, ws_size, arma.fill.zeros)
        vs = us

        N = d**2.0
        for i in range(hs_size - 1):
            for j in range(ws_size - 1):
                hk = i * d
                hl = (i + 1) * d - 1
                wk = j * d
                wl = (j + 1) * d - 1
                sum1 = arma.sum(arma.sum(u[hk:hl, wk:wl]))[0, 0]
                sum2 = arma.sum(arma.sum(v[hk:hl, wk:wl]))[0, 0]
                us[i, j] = sum1 / N
                vs[i, j] = sum2 / N
        flow.append(us)
        flow.append(vs)
        out.append(flow)
        flow = [] 
    return out


## Wrapper Functions

In [47]:
def make_db(path, id):
    seq = make_sequence(path)
    #id = str(path).split('\\')[-1]
    flow = findOpticalflow(seq)
    vecs = findFlowUtil(flow)
    write_vectors(vecs, id)

In [34]:
def match_person(pseq_path, gallery_path):
    probe_seq = make_sequence(pseq_path)
    probe_flow = findOpticalflow(probe_seq)
    probe_vecs = findFlowUtil(probe_flow)
    distances = []
    for i in os.scandir(gallery_path):
        if i.is_dir():
            curr_subject = []
            curr_subject.append(read_list(gallery_path + '//' + i.name + '//usRowW.txt'))
            curr_subject.append(read_list(gallery_path + '//' + i.name + '//usColW.txt'))
            curr_subject.append(read_list(gallery_path + '//' + i.name + '//vsRowW.txt'))
            curr_subject.append(read_list(gallery_path + '//' + i.name + '//vsColW.txt'))
            dist = calcDistance(curr_subject, probe_vecs)
            distances.append((dist, i.name))
    return min(distances, key=lambda x: x[0])[1]



## Testing CASIA

In [20]:
sub1 = "D:\\Dataset\\CASIA\\001\\nm-01\\090"
sub2 = "D:\\Dataset\\CASIA\\002\\nm-01\\090"
sub3 = "D:\\Dataset\\CASIA\\003\\nm-01\\090"
sub4 = "D:\\Dataset\\CASIA\\004\\nm-01\\090"
sub5 = "D:\\Dataset\\CASIA\\005\\nm-01\\090"

In [23]:
make_db(sub1,1)

In [26]:
make_db(sub2, 2)
make_db(sub3, 3)
make_db(sub4, 4)
make_db(sub5, 5)

In [35]:
match_person(sub2, "Gallery")

'2'

## Testing Osaka

In [48]:
s1 = "D:\\Dataset\\Silhouette_090-00\\00001"
s2 = "D:\\Dataset\\Silhouette_090-00\\00002"
s3 = "D:\\Dataset\\Silhouette_090-00\\00003"
s4 = "D:\\Dataset\\Silhouette_090-00\\00004"
s5 = "D:\\Dataset\\Silhouette_090-00\\00005"

In [50]:
make_db(s1,1)

In [51]:
make_db(s2,2)
make_db(s3,3)
make_db(s4,4)
make_db(s5,5)

In [52]:
match_person(s3, "Gallery")

'3'